In [ ]:
import json
import pandas as pd
import time
from datetime import datetime
from IPython.display import display, Markdown
import ipywidgets as widgets

import importlib
import sys
# 动态添加当前工作目录到sys.path，如果还没有添加的话
import os
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)
# 尝试重新加载模块，首次运行时忽略错误
try:
    import clientprocessor
    importlib.reload(clientprocessor)  # 如果模块已经存在，则重新加载
except ImportError:
    import clientprocessor  # 第一次导入模块
# 从重新加载（或初次加载）的模块中导入 ClientProcessor 类
from clientprocessor import ClientProcessor


clientreq = ""
button_info = widgets.Button(description = "info", disabled = False, button_style = "info", icon = 'check')
button_primary = widgets.Button(description = "primary", disabled = False, button_style = "primary", icon = 'times')
button_warning = widgets.Button(description = "warning", disabled = False, button_style = "warning")
button_danger = widgets.Button(description = "danger", disabled = False, button_style = "danger")

wind_input_text = widgets.Text(placeholder = "输入全量日志中的请求id", description = "原始请求ID", disabled = False)
wind_button_raw_request  = widgets.Button(description = "请求详情", disabled = False, button_style = "success")
wind_button_clear  = widgets.Button(description = "清除", disabled = False, button_style = "warning")

#*******************************************************************************    
wind_head = widgets.Label(value = "日志分析")
wind_fileupload = widgets.FileUpload(accept = ".log", description = "1) 上传日志文件", disabled=False, multiple = True, layout=widgets.Layout(width='170px'))
wind_button_analyse = widgets.Button(description = "2) 日志解析", disabled = False, button_style = "success")
button_init = widgets.Button(description = "3) 初始化(点一下!)", disabled = False, button_style = "primary")
button_summary = widgets.Button(description = "4) 请求统计(点一下!)", disabled = False, button_style = "primary")
# 设置控件
checkbox_unlimit_table_length = widgets.Checkbox(value=False, description='不限最大行数(默认50)', disabled=False)
# 账户查询控件
button_account = widgets.Button(description = "账户信息", disabled = False, button_style = "success")
dropdown_querytime_account = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
button_clear_account = widgets.Button(description = "清除账户信息", disabled = False, button_style = "warning")
# 资金查询控件
button_fund = widgets.Button(description = "资金信息", disabled = False, button_style = "success")
dropdown_fundtoken = widgets.Dropdown(options = [], value = None, description = "fund_key", disabled = False)
button_clear_fund = widgets.Button(description = "清除资金信息", disabled = False, button_style = "warning")
# 持仓查询控件
button_position = widgets.Button(description = "持仓信息", disabled = False, button_style = "success")
button_clear_position = widgets.Button(description = "清除持仓信息", disabled = False, button_style = "warning")
dropdown_fund_position = widgets.Dropdown(options = [], value = None, description = "fund_key", disabled = False)
dropdown_querytime_position = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
# 委托查询控件
button_queryorder = widgets.Button(description = "委托信息(先点)", disabled = False, button_style = "success")
button_clear_queryorder = widgets.Button(description = "清除委托信息", disabled = False, button_style = "warning")
dropdown_fund_queryorder = widgets.Dropdown(options = [], value = None, description = "fund_key", disabled = False)
dropdown_querytime_queryorder = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
button_queryorder_summary = widgets.Button(description = "单账户委托汇总", disabled = False, button_style = "success")
button_queryorder_summary_all = widgets.Button(description = "全账户委托汇总", disabled = False, button_style = "success")
# 成交查询控件
button_querytrade = widgets.Button(description = "成交信息(先点)", disabled = False, button_style = "success")
button_clear_querytrade = widgets.Button(description = "清除成交信息", disabled = False, button_style = "warning")
dropdown_fund_querytrade = widgets.Dropdown(options = [], value = None, description = "fund_key", disabled = False)
dropdown_querytime_querytrade = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
button_querytrade_summary = widgets.Button(description = "单账户成交汇总", disabled = False, button_style = "success")
button_querytrade_summary_all = widgets.Button(description = "全账户成交汇总", disabled = False, button_style = "success")
checkbox_querytrade_stockfilter = widgets.Checkbox(value=False, description='代码筛选', disabled=False, layout=widgets.Layout(width='180px'))
input_code_querytrade = widgets.Text(placeholder = "筛选股票代码", description = "证券代码", disabled = False, layout=widgets.Layout(width='250px'))

# 查询可融资标的券列表控件
button_501005 = widgets.Button(description = "可融资标的券", disabled = False, button_style = "success")
button_clear_501005 = widgets.Button(description = "清除查询信息", disabled = False, button_style = "warning")
dropdown_fund_501005 = widgets.Dropdown(options = [], value = None, description = "fund_key", disabled = False)
dropdown_querytime_501005 = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
input_code_501005 = widgets.Text(placeholder = "筛选股票代码", description = "证券代码", disabled = False)
checkbox_501005 = widgets.Checkbox(value=False, description='展示全部股票', disabled=False,
                                   style={'description_width': '0px'},
                                   layout=widgets.Layout(width='auto', margin='0px 0px 0px 0px'))
# 集中/篮子交易控件
button_basket = widgets.Button(description = "集中/篮子交易", disabled = False, button_style = "success")
input_basket_instanceid = widgets.Text(placeholder = "筛选母单instance_id", description = "母单", disabled = False)
input_code_basket = widgets.Text(placeholder = "筛选交易过的股票代码", description = "证券代码", disabled = False, layout=widgets.Layout(width='250px'))
input_fund_basket = widgets.Text(placeholder = "筛选交易过的资金账号", description = "资金账号", disabled = False, layout=widgets.Layout(width='250px'))
dropdown_basket_orderid = widgets.Dropdown(options = [], value = None, description = "子单", disabled = False)
button_basket_filterinstance = widgets.Button(description = "筛选母单", disabled = False, button_style = "success", layout=widgets.Layout(width='100px'))
button_basket_filterorder = widgets.Button(description = "筛选子单", disabled = False, button_style = "success", layout=widgets.Layout(width='100px'))
button_basket_instance_detail = widgets.Button(description = "母单详情", disabled = False, button_style = "success")
button_basketpush = widgets.Button(description = "当前子单详情", disabled = False, button_style = "success")
button_basketpush_all = widgets.Button(description = "全部子单详情", disabled = False, button_style = "success")
button_clear_basket = widgets.Button(description = "清除母单列表", disabled = False, button_style = "warning")
button_clear_basket_all = widgets.Button(description = "清除全部", disabled = False, button_style = "warning")
# 算法交易控件
button_algorithm = widgets.Button(description = "算法交易", disabled = False, button_style = "success")
input_algorithm_instanceid = widgets.Text(placeholder = "筛选母单instance_id", description = "母单", disabled = False)
button_algorithm_instance_detail = widgets.Button(description = "母单详情", disabled = False, button_style = "success")
button_algorithm_instance_push = widgets.Button(description = "母单推送信息", disabled = False, button_style = "success")
button_algorithm_order_push = widgets.Button(description = "委托推送信息", disabled = False, button_style = "success")
button_clear_algorithm = widgets.Button(description = "清除母单列表", disabled = False, button_style = "warning")
button_clear_algorithm_push = widgets.Button(description = "清除推送列表", disabled = False, button_style = "warning")
button_clear_algorithm_all = widgets.Button(description = "清除全部", disabled = False, button_style = "warning")
dropdown_querytime_algorithm = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
button_query_algorithm = widgets.Button(description = "查询结果", disabled = False, button_style = "success")
# 条件交易控件
button_condition = widgets.Button(description = "条件交易", disabled = False, button_style = "success")
input_condition_instanceid = widgets.Text(placeholder = "筛选母单instance_id", description = "母单", disabled = False)
input_condition_fund = widgets.Text(placeholder = "筛选资金账号", description = "资金账号", disabled = False,
                                                style={'description_width': '60px'},
                                                layout=widgets.Layout(width='180px', margin='0px 0px 0px 0px'))
input_condition_stockcode = widgets.Text(placeholder = "筛选证券代码", description = "证券代码", disabled = False,
                                                style={'description_width': '60px'},
                                                layout=widgets.Layout(width='170px', margin='0px 0px 0px 0px'))
button_condition_filterinstance = widgets.Button(description = "筛选母单", disabled = False, button_style = "success", layout=widgets.Layout(width='150px'))
button_condition_filterorder = widgets.Button(description = "筛选子单", disabled = False, button_style = "success", layout=widgets.Layout(width='150px'))
button_initreqs_condition = widgets.Button(description = "原始请求", disabled = False, button_style = "success")
checkbox_fullinitreqs_condition = widgets.Checkbox(value=False, description='展示请求原始长度', disabled=False,
                                                style={'description_width': '0px'},
                                                layout=widgets.Layout(width='auto', margin='0px 0px 0px 0px'))
button_clear_initreqs_condition = widgets.Button(description = "清除原始请求", disabled = False, button_style = "warning")
button_condition_instance_detail = widgets.Button(description = "条件母单详情", disabled = False, button_style = "success")
button_condition_order_detail = widgets.Button(description = "条件子单详情", disabled = False, button_style = "success")
button_condition_security_order_detail = widgets.Button(description = "条件子单个股详情", disabled = False, button_style = "success")
button_clear_condition_all = widgets.Button(description = "清除全部", disabled = False, button_style = "warning")
dropdown_querytime_condition = widgets.Dropdown(options = [], value = None, description = "查询时间", disabled = False)
button_query_condition = widgets.Button(description = "查询结果", disabled = False, button_style = "success")

empty_output = widgets.Output(layout=widgets.Layout(width='150px'))
output_err = widgets.Output()
output_request_detail = widgets.Output()
output_summary = widgets.Output()
output_account = widgets.Output()
output_fundtoken_dict = widgets.Output()
output_fund_dict = widgets.Output()
output_fund = widgets.Output()
output_position = widgets.Output()
output_queryorder = widgets.Output()
output_querytrade = widgets.Output()
output_501005 = widgets.Output()
output_basket = widgets.Output()
output_basketfilter = widgets.Output()
# output_basketpush = widgets.Output(layout=widgets.Layout(height='600px'))
output_basketpush = widgets.Output()
output_algorithm = widgets.Output()
output_algorithm_filter = widgets.Output()
output_algorithm_instance_push = widgets.Output()
output_algorithm_order_push = widgets.Output()
output_condition_initreqs = widgets.Output()
output_condition = widgets.Output()
output_condition_instance_detail = widgets.Output()
output_condition_order_detail = widgets.Output()
output_condition_order_security_detail = widgets.Output()

#*******************************************************************************    
        
def on_wind_button_raw_request(b):
    req_id = wind_input_text.value.strip()
    output_request_detail.clear_output()
    with output_request_detail:
        clientreq.show_request_and_response(req_id)


def on_wind_button_clear(b):
    wind_input_text.value = ""
    output_request_detail.clear_output()
    
    
def on_button_summary(b):
    output_summary.clear_output()
    with output_summary:
        print('userid:%s, username:%s' % (clientreq.userid, json.dumps(clientreq.username)))
        print("日志起始时间:%s" % clientreq.log_begin_time)
        print("日志结束时间:%s" % clientreq.log_end_time)
        clientreq.show_request_statics()

#*******************************************************************************

def on_wind_button_analyse(b):
    output_err.clear_output()
    with output_err:
        if not wind_fileupload.value:
            return print("error:", "请上传日志文件")
        tmpfile = "tmp.log"
        stime = time.perf_counter()
        with open(tmpfile, "wb") as f:
            for file in wind_fileupload.value:
                print("加载文件", file.name, "文件大小", len(file.content))
                f.write(file.content)
        try:
            global clientreq
            clientreq = ClientProcessor([tmpfile])
            clientreq.parse()
            print("解析完成", "解析失败日志行数:", len(clientreq.illegal_reqs))
            print("跳过不处理的日志行数:", len(clientreq.skipped_reqs))
        except Exception as e:
            print("解析出错", e)
        usetime = time.perf_counter() - stime
        print("处理耗时:", usetime)


def on_button_init(b):
    output_fundtoken_dict.clear_output()
    output_fund_dict.clear_output()
    clientreq.handle_account_query()
    clientreq.handle_basket_order_push()
    clientreq.handle_algorithm_push()
    with output_fundtoken_dict:
        display("fundtoken_dict", clientreq.fundtoken_dict)
        display("*"*50)
        display("client_fundtoken_mapping", clientreq.client_fundtoken_mapping)
        display("*"*50)
        dict_permissioncode = {"普通股票（沪深A股）":1, "港股通":2, "大宗交易":4, "融资计算":8}
        display("client_permissioncode_mapping", dict_permissioncode, clientreq.client_permissioncode_mapping)
        display("*"*50)
    with output_fund_dict:
        display("fund_dict", clientreq.fund_dict)


def on_button_account(b):
    dropdown_querytime_account.options = []
    dropdown_querytime_account.value = None
    output_account.clear_output()
    with output_account:
        options = [req_time for req_time in clientreq.query_accounts_df]
        dropdown_querytime_account.options = options
        dropdown_querytime_account.value = options[0]

        
def on_dropdown_querytime_account_change(change):
    output_account.clear_output()
    with output_account:
        selected_value = change["new"]
        print(selected_value)
        df_user_account = clientreq.handle_account_query(selected_value)
        display(df_user_account)


def on_button_clear_account(b):
    dropdown_querytime_account.options = []
    dropdown_querytime_account.value = None
    output_account.clear_output()
    output_fundtoken_dict.clear_output()
    output_fund_dict.clear_output()


def on_button_fund(b):
    dropdown_fundtoken.options = []
    dropdown_fundtoken.value = None
    output_fund.clear_output()
    with output_fund:
        clientreq.handle_fund_query()
        options = [key for key in clientreq.query_fund_df]
        dropdown_fundtoken.options = options
        dropdown_fundtoken.value = options[0]


def on_dropdown_fundtoken_change(change):
    output_fund.clear_output()
    with output_fund:
        selected_value = change["new"]
        print(selected_value)
        df_query_fund = clientreq.query_fund_df[selected_value]
        display(df_query_fund)


def on_button_clear_fund(b):
    dropdown_fundtoken.options = []
    dropdown_fundtoken.value = None
    output_fund.clear_output()


def on_button_position(b):
    dropdown_fund_position.options = []
    dropdown_fund_position.value = None
    output_position.clear_output()
    with output_position:
        clientreq.handle_position_query()
        options = [key for key in clientreq.query_position_df.keys()]
        dropdown_fund_position.options = options
        dropdown_fund_position.value = options[0]


def on_dropdown_fund_position_change(change):
    dropdown_querytime_position.options = []
    dropdown_querytime_position.value = None
    output_position.clear_output()
    with output_position:
        selected_account = dropdown_fund_position.value
        options = [key for key in clientreq.query_position_df[selected_account].keys()]
        resorted_options = [item for item in sorted(options, key=lambda item: datetime.strptime(item.split('|')[0], '%Y%m%d %H:%M:%S.%f'), reverse=True)]
        dropdown_querytime_position.options = resorted_options
        dropdown_querytime_position.value = resorted_options[0]


def on_dropdown_querytime_position_change(change):
    output_position.clear_output()
    with output_position:
        selected_account = dropdown_fund_position.value
        selected_querytime = dropdown_querytime_position.value
        if selected_querytime:
            print(selected_account, selected_querytime)
            clientreq.show_queryposition(selected_account, selected_querytime)


def on_button_clear_position(b):
    dropdown_fund_position.options = []
    dropdown_fund_position.value = None
    dropdown_querytime_position.options = []
    dropdown_querytime_position.value = None
    output_position.clear_output()


def on_button_queryorder(b):
    dropdown_fund_queryorder.options = []
    dropdown_fund_queryorder.value = None
    output_queryorder.clear_output()
    with output_queryorder:
        clientreq.handle_order_query()
        options = [key for key in clientreq.query_order_df.keys()]
        dropdown_fund_queryorder.options = options
        dropdown_fund_queryorder.value = options[0]

def on_dropdown_fund_queryorder_change(change):
    dropdown_querytime_queryorder.options = []
    dropdown_querytime_queryorder.value = None
    output_queryorder.clear_output()
    with output_queryorder:
        selected_account = dropdown_fund_queryorder.value
        options = [key for key in clientreq.query_order_df[selected_account].keys()]
        resorted_options = [item for item in sorted(options, key=lambda item: datetime.strptime(item.split('|')[0], '%Y%m%d %H:%M:%S.%f'), reverse=True)]
        dropdown_querytime_queryorder.options = resorted_options
        dropdown_querytime_queryorder.value = resorted_options[0]


def on_dropdown_querytime_queryorder_change(change):
    output_queryorder.clear_output()
    with output_queryorder:
        selected_account = dropdown_fund_queryorder.value
        selected_querytime = dropdown_querytime_queryorder.value
        if selected_querytime:
            print(selected_account, selected_querytime)
            clientreq.show_queryorder(selected_account, selected_querytime)


def on_button_clear_queryorder(b):
    dropdown_fund_queryorder.options = []
    dropdown_fund_queryorder.value = None
    dropdown_querytime_queryorder.options = []
    dropdown_querytime_queryorder.value = None
    output_queryorder.clear_output()

def on_button_queryorder_summary(b):
    output_queryorder.clear_output()
    with output_queryorder:
        selected_account = dropdown_fund_queryorder.value
        print("查询账号：", selected_account)
        for query_time, query_data in clientreq.query_order_df[selected_account].items():
            print(query_time, len(query_data))


def on_button_queryorder_summary_all(b):
    output_queryorder.clear_output()
    with output_queryorder:
        query_order_df = clientreq.query_order_df
        column_index = set()
        row_index = set()
        for fundkey, querydata in query_order_df.items():
            for timekey, df_querydata in querydata.items():
                timestamp = timekey.split(' ')[1].split('.')[0]
                column_index.add(fundkey)
                row_index.add(timestamp)
        
        column_index = list(column_index)
        row_index = sorted(list(row_index))
        
        df = pd.DataFrame(index=row_index, columns=column_index)
        for fundkey, querydata in query_order_df.items():
            for timekey, df_querydata in querydata.items():
                timestamp = timekey.split(' ')[1].split('.')[0]
                df.loc[timestamp, fundkey] = len(df_querydata)
        
        df = df.fillna(0)
        df = df.loc[:, (df != 0).any(axis=0)]
        df['total'] = df.sum(axis=1)
        
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]  # 将 'total' 列移到最前面
        df = df[cols]
        display(df)


def on_button_querytrade(b):
    dropdown_fund_querytrade.options = []
    dropdown_fund_querytrade.value = None
    output_querytrade.clear_output()
    with output_querytrade:
        clientreq.handle_trade_query()
        options = [key for key in clientreq.query_trade_df.keys()]
        dropdown_fund_querytrade.options = options
        dropdown_fund_querytrade.value = options[0]


def on_dropdown_fund_querytrade_change(change):
    dropdown_querytime_querytrade.options = []
    dropdown_querytime_querytrade.value = None
    output_querytrade.clear_output()
    with output_querytrade:
        selected_account = dropdown_fund_querytrade.value
        options = [key for key in clientreq.query_trade_df[selected_account].keys()]
        resorted_options = [item for item in sorted(options, key=lambda item: datetime.strptime(item.split('|')[0], '%Y%m%d %H:%M:%S.%f'), reverse=True)]
        dropdown_querytime_querytrade.options = resorted_options
        dropdown_querytime_querytrade.value = resorted_options[0]


def on_dropdown_querytime_querytrade_change(change):
    output_querytrade.clear_output()
    filter_code = ""
    if checkbox_querytrade_stockfilter.value:
        filter_code = input_code_querytrade.value
    with output_querytrade:
        selected_account = dropdown_fund_querytrade.value
        selected_querytime = dropdown_querytime_querytrade.value
        if selected_querytime:
            clientreq.show_querytrade(selected_account, selected_querytime, filter_code)


def on_button_clear_querytrade(b):
    dropdown_fund_querytrade.options = []
    dropdown_fund_querytrade.value = None
    dropdown_querytime_querytrade.options = []
    dropdown_querytime_querytrade.value = None
    output_querytrade.clear_output()
    checkbox_querytrade_stockfilter.value = False
    input_code_querytrade.value = ''
    
def on_button_querytrade_summary(b):
    output_querytrade.clear_output()
    with output_querytrade:
        selected_account = dropdown_fund_querytrade.value
        print("查询账号：", selected_account)
        for query_time, query_data in clientreq.query_trade_df[selected_account].items():
            print(query_time, len(query_data))

            
def on_button_querytrade_summary_all(b):
    output_querytrade.clear_output()
    with output_querytrade:
        query_trade_df = clientreq.query_trade_df
        column_index = set()
        row_index = set()
        for fundkey, querydata in query_trade_df.items():
            for timekey, df_querydata in querydata.items():
                timestamp = timekey.split(' ')[1].split('.')[0]
                column_index.add(fundkey)
                row_index.add(timestamp)
        
        column_index = list(column_index)
        row_index = sorted(list(row_index))
        
        df = pd.DataFrame(index=row_index, columns=column_index)
        for fundkey, querydata in query_trade_df.items():
            for timekey, df_querydata in querydata.items():
                timestamp = timekey.split(' ')[1].split('.')[0]
                df.loc[timestamp, fundkey] = len(df_querydata)
        
        df = df.fillna(0)
        df = df.loc[:, (df != 0).any(axis=0)]
        df['total'] = df.sum(axis=1)
        
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]  # 将 'total' 列移到最前面
        df = df[cols]
        display(df)


def on_button_501005(b):
    dropdown_fund_501005.options = []
    dropdown_fund_501005.value = None
    output_501005.clear_output()
    with output_501005:
        clientreq.handle_finable_security_query()
        options = [key for key in clientreq.query_finable_security_dict.keys()]
        dropdown_fund_501005.options = options
        dropdown_fund_501005.value = options[0]


def on_dropdown_fund_501005_change(change):
    dropdown_querytime_501005.options = []
    dropdown_querytime_501005.value = None
    output_501005.clear_output()
    with output_501005:
        selected_account = dropdown_fund_501005.value
        options = [key for key in clientreq.query_finable_security_dict[selected_account].keys()]
        resorted_options = [item for item in sorted(options, key=lambda item: datetime.strptime(item.split('|')[0], '%Y%m%d %H:%M:%S.%f'), reverse=True)]
        dropdown_querytime_501005.options = resorted_options
        dropdown_querytime_501005.value = resorted_options[0]


def on_dropdown_querytime_501005(change):
    output_501005.clear_output()
    with output_501005:
        selected_account = dropdown_fund_501005.value
        selected_querytime = dropdown_querytime_501005.value
        show_detail = checkbox_501005.value
        filter_stockcode = input_code_501005.value
        clientreq.show_finable_security(selected_account, selected_querytime, show_detail, filter_stockcode)


def on_checkbox_501005(change):
    output_501005.clear_output()
    with output_501005:
        selected_account = dropdown_fund_501005.value
        selected_querytime = dropdown_querytime_501005.value
        show_detail = checkbox_501005.value
        filter_stockcode = input_code_501005.value
        clientreq.show_finable_security(selected_account, selected_querytime, show_detail, filter_stockcode)


def on_button_clear_501005(b):
    dropdown_fund_501005.options = []
    dropdown_fund_501005.value = None
    dropdown_querytime_501005.options = []
    dropdown_querytime_501005.value = None
    input_code_501005.value = ''
    checkbox_501005.value = False
    output_501005.clear_output()


def on_button_basket(b):
    dropdown_basket_orderid.options = []
    dropdown_basket_orderid.value = None
    input_basket_instanceid.value = ''
    input_code_basket.value = ''
    output_basket.clear_output()
    output_basketfilter.clear_output()
    output_basketpush.clear_output()
    with output_basket:
        df_basketorder = clientreq.handle_basketorder()
        security_list = clientreq.get_basketorder_code()
        print("涉及交易的股票列表", security_list)
        display("母单列表", df_basketorder)


def on_button_basket_filterinstance(b):
    if input_code_basket.value != "":
        stockcode = input_code_basket.value.strip()
        dropdown_basket_orderid.options = []
        dropdown_basket_orderid.value = None
        input_basket_instanceid.value = ''
        output_basket.clear_output()
        output_basketfilter.clear_output()
        output_basketpush.clear_output()
        with output_basket:
            df_basketorder = clientreq.handle_basketorder(stockcode)
            security_list = clientreq.get_basketorder_code()
            print("涉及交易的股票列表", security_list)
            display("筛选母单列表", df_basketorder)
    else:
        on_button_basket(b)

        
def on_button_basket_filterorder(b):
    if input_code_basket.value != "" or input_fund_basket.value != "":
        stockcode = input_code_basket.value.strip()
        fund = input_fund_basket.value.strip()
        dropdown_basket_orderid.options = []
        dropdown_basket_orderid.value = None
        input_basket_instanceid.value = ''
        output_basket.clear_output()
        output_basketfilter.clear_output()
        output_basketpush.clear_output()
        with output_basket:
            df_basketorder = clientreq.handle_basketorder(stockcode)
            security_list = clientreq.get_basketorder_code()
            print("涉及交易的股票列表", security_list)
            display("筛选母单列表", df_basketorder)
            for instanceid in df_basketorder["instanceid"]:
                df_instance_info = df_basketorder[df_basketorder["instanceid"] == instanceid]
                display("母单信息", instanceid, df_instance_info)
                df_basketorder_detail = clientreq.get_basketorder_detail(instanceid)
                display("母单委托信息", df_basketorder_detail)
                fundkey_list = df_basketorder_detail.apply(lambda row:row["fund_token"]+"|"+row["SecurityID"], axis=1).tolist()
                for fund_key in fundkey_list:
                    fund_token = fund_key.split('|')[0]
                    if fund in clientreq.fundtoken_dict[fund_token]:
                        df_basketorder_push = clientreq.get_basketorder_push_detail(instanceid, fund_key)
                        print("子单详情", fund_key, clientreq.fundtoken_dict[fund_token])
                        print("text_list", list(set(df_basketorder_push["Text"].tolist())))
                        display(df_basketorder_push)
                print("#"*120)
    else:
        on_button_basket(b)


def on_button_basket_instance_detail(b):
    dropdown_basket_orderid.options = []
    dropdown_basket_orderid.value = None
    output_basketfilter.clear_output()
    with output_basketfilter:
        instanceid = input_basket_instanceid.value.strip()
        if instanceid == "":
            print("母单id为空，请输入母单id筛选母单")
        else:
            df_basketorder = clientreq.handle_basketorder()
            if not (df_basketorder["instanceid"] == instanceid).any():
                print("母单id[%s]不存在，请核对后再次输入" % instanceid)
            else:
                df_instance_info = df_basketorder[df_basketorder["instanceid"] == instanceid]
                display("母单信息", instanceid, df_instance_info)
                df_basketorder_detail = clientreq.get_basketorder_detail(instanceid)
                display("母单委托信息", df_basketorder_detail)
                options = df_basketorder_detail.apply(lambda row:row["fund_token"]+"|"+row["SecurityID"], axis=1).tolist()
                dropdown_basket_orderid.options = options
                dropdown_basket_orderid.value = options[0]


def on_dropdown_basket_orderid_change(change):
    pass


def on_button_basketpush(b):
    output_basketpush.clear_output()
    with output_basketpush:
        fund_key = dropdown_basket_orderid.value
        instanceid = input_basket_instanceid.value.strip()
        if fund_key != "" and instanceid != "":
            fund_token = fund_key.split('|')[0]
            df_basketorder_push = clientreq.get_basketorder_push_detail(instanceid, fund_key)
            print("子单详情", fund_key, clientreq.fundtoken_dict[fund_token])
            print("text_list", list(set(df_basketorder_push["Text"].tolist())))
            display(df_basketorder_push)


def on_button_basketpush_all(b):
    output_basketpush.clear_output()
    with output_basketpush:
        instanceid = input_basket_instanceid.value.strip()
        for fund_key in dropdown_basket_orderid.options:
            fund_token = fund_key.split('|')[0]
            df_basketorder_push = clientreq.get_basketorder_push_detail(instanceid, fund_key)
            print("子单详情", fund_key, clientreq.fundtoken_dict[fund_token])
            print("text_list", list(set(df_basketorder_push["Text"].tolist())))
            display(df_basketorder_push)


def on_button_clear_basket(b):
    output_basket.clear_output()


def on_button_clear_basket_all(b):
    dropdown_basket_orderid.options = []
    dropdown_basket_orderid.value = None
    input_basket_instanceid.value = ''
    input_code_basket.value = ''
    input_fund_basket.value = ''
    output_basket.clear_output()
    output_basketfilter.clear_output()
    output_basketpush.clear_output()


def on_button_algorithm(b):
    input_algorithm_instanceid.value = ""
    output_algorithm.clear_output()
    output_algorithm_filter.clear_output()
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()
    dropdown_querytime_algorithm.options = []
    dropdown_querytime_algorithm.value = None
    with output_algorithm:
        df_algorithm = clientreq.handle_algorithm()
        security_list = clientreq.get_algorithm_code()
        print("涉及交易的股票列表", security_list)
        display("筛选母单列表", df_algorithm)


def on_button_algorithm_instance_detail(b):
    output_algorithm_filter.clear_output()
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()
    with output_algorithm_filter:
        instanceid = input_algorithm_instanceid.value.strip()
        if instanceid == "":
            print("母单id为空，请输入母单id")
        else:
            df_algorithm = clientreq.handle_algorithm()
            if not (df_algorithm["inst_id"] == instanceid).any():
                print("母单id[%s]不存在，请核对后再次输入" % instanceid)
            else:
                df_instance_info = df_algorithm[df_algorithm["inst_id"] == instanceid]
                display("母单信息", instanceid, df_instance_info)
                df_algorithm_detail = clientreq.get_algorithm_detail(instanceid)
                display("母单参数信息", df_algorithm_detail)


def on_button_algorithm_instance_push(b):
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()
    with output_algorithm_instance_push:
        instanceid = input_algorithm_instanceid.value.strip()
        if instanceid == "":
            print("母单id为空, 请输入母单id")
        else:
            push_key, df_algorithm_push = clientreq.get_algorithm_push_detail(instanceid, "instruction")
            print("推送类型:" + push_key, " msg_list", list(set(df_algorithm_push["msg"].tolist())))
            display(df_algorithm_push)


def on_button_algorithm_order_push(b):
    output_algorithm_order_push.clear_output()
    with output_algorithm_instance_push:
        instanceid = input_algorithm_instanceid.value.strip()
        if instanceid == "":
            print("母单id为空，请输入母单id")
        else:
            push_key, df_algorithm_push = clientreq.get_algorithm_push_detail(instanceid, "order")
            print("推送类型:" + push_key, " msg_list", list(set(df_algorithm_push["msg"].tolist())))
            display(df_algorithm_push)


def on_button_query_algorithm(b):
    input_algorithm_instanceid.value = ""
    output_algorithm.clear_output()
    output_algorithm_filter.clear_output()
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()
    dropdown_querytime_algorithm.options = []
    dropdown_querytime_algorithm.value = None
    with output_algorithm:
        clientreq.handle_algorithm()
        options = [key for key in clientreq.query_algorithm_df.keys()]
        dropdown_querytime_algorithm.options = options
        dropdown_querytime_algorithm.value = options[0]


def on_dropdown_querytime_algorithm_change(change):
    output_algorithm.clear_output()
    with output_algorithm:
        selected_querytime = dropdown_querytime_algorithm.value
        if selected_querytime:
            clientreq.show_queryalgorithm(selected_querytime)


def on_button_clear_algorithm(b):
    input_algorithm_instanceid.value = ""
    output_algorithm.clear_output()
    output_algorithm_filter.clear_output()


def on_button_clear_algorithm_push(b):
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()


def on_button_clear_algorithm_all(b):
    input_algorithm_instanceid.value = ""
    output_algorithm.clear_output()
    output_algorithm_filter.clear_output()
    output_algorithm_instance_push.clear_output()
    output_algorithm_order_push.clear_output()
    dropdown_basket_orderid.options = []
    dropdown_basket_orderid.value = None


def on_button_condition(b):
    on_button_clear_condition_all(b)
    with output_condition:
        clientreq.handle_condition()
        clientreq.show_condition_summary()


def on_button_condition_filterinstance(b):
    # output_condition.clear_output():
    with output_condition:
        print("筛选母单操作")


def on_button_condition_filterorder(b):
    with output_condition:
        print("筛选母单操作")
    

def on_button_initreqs_condition(b):
    output_condition_initreqs.clear_output()
    with output_condition_initreqs:
        showfullreqs = checkbox_fullinitreqs_condition.value
        instanceid = input_condition_instanceid.value.strip()
        clientreq.show_condition_initreqs(instanceid, showfullreqs)


def on_button_clear_initreqs_condition(b):
    output_condition_initreqs.clear_output()


def on_checkbox_fullinitreqs_condition(change):
    output_condition_initreqs.clear_output()
    with output_condition_initreqs:
        showfullreqs = checkbox_fullinitreqs_condition.value
        instanceid = input_condition_instanceid.value.strip()
        clientreq.show_condition_initreqs(instanceid, showfullreqs)


def on_button_condition_instance_detail(b):
    output_condition_instance_detail.clear_output()
    with output_condition_instance_detail:
        instanceid = input_condition_instanceid.value.strip()
        fund = input_condition_fund.value.strip()
        stockcode = input_condition_stockcode.value.strip()
        clientreq.show_condition_instance_detail(instanceid, fund, stockcode)


def on_button_condition_order_detail(b):
    output_condition_order_detail.clear_output()
    with output_condition_order_detail:
        instanceid = input_condition_instanceid.value.strip()
        clientreq.show_condition_order_detail(instanceid)


def on_button_condition_security_order_detail(b):
    output_condition_order_security_detail.clear_output()
    with output_condition_order_security_detail:
        instanceid = input_condition_instanceid.value.strip()
        fund = input_condition_fund.value.strip()
        stockcode = input_condition_stockcode.value.strip()
        clientreq.show_condition_security_order_detail(instanceid, fund, stockcode)

    
def on_button_query_condition(b):
    dropdown_querytime_condition.options = []
    dropdown_querytime_condition.value = None
    output_condition.clear_output()
    with output_condition:
        clientreq.handle_condition_query()
        options = [key for key in clientreq.query_gradecondition_df.keys()]
        dropdown_querytime_condition.options = options
        dropdown_querytime_condition.value = options[0]


def on_dropdown_querytime_condition_change(change):
    output_condition.clear_output()
    with output_condition:
        selected_querytime = dropdown_querytime_condition.value
        if selected_querytime:
            clientreq.show_querycondition(selected_querytime)


def on_button_clear_condition_all(b):
    input_condition_instanceid.value = ""
    input_condition_fund.value = ""
    input_condition_stockcode.value = ""
    dropdown_querytime_condition.options = []
    dropdown_querytime_condition.value = None
    output_condition.clear_output()
    output_condition_initreqs.clear_output()
    output_condition_instance_detail.clear_output()
    output_condition_order_detail.clear_output()
    output_condition_order_security_detail.clear_output()


def on_checkbox_unlimit_table_length(change):
    is_unlimited = checkbox_unlimit_table_length.value
    if is_unlimited:
        pd.set_option('display.max_rows', None)
    else:
        pd.set_option('display.max_rows', 50)


#*******************************************************************************

wind_button_analyse.on_click(on_wind_button_analyse)
wind_button_raw_request.on_click(on_wind_button_raw_request)
wind_button_clear.on_click(on_wind_button_clear)
button_init.on_click(on_button_init)
button_summary.on_click(on_button_summary)

# 查账户列表
button_account.on_click(on_button_account)
dropdown_querytime_account.observe(on_dropdown_querytime_account_change, names = "value")
button_clear_account.on_click(on_button_clear_account)
# 查资金信息
button_fund.on_click(on_button_fund)
dropdown_fundtoken.observe(on_dropdown_fundtoken_change, names = "value")
button_clear_fund.on_click(on_button_clear_fund)
# 查持仓信息
button_position.on_click(on_button_position)
dropdown_fund_position.observe(on_dropdown_fund_position_change, names = "value")
dropdown_querytime_position.observe(on_dropdown_querytime_position_change, names = "value")
button_clear_position.on_click(on_button_clear_position)
# 查委托信息
button_queryorder.on_click(on_button_queryorder)
dropdown_fund_queryorder.observe(on_dropdown_fund_queryorder_change, names = "value")
dropdown_querytime_queryorder.observe(on_dropdown_querytime_queryorder_change, names = "value")
button_clear_queryorder.on_click(on_button_clear_queryorder)
button_queryorder_summary.on_click(on_button_queryorder_summary)
button_queryorder_summary_all.on_click(on_button_queryorder_summary_all)
# 查成交信息
button_querytrade.on_click(on_button_querytrade)
dropdown_fund_querytrade.observe(on_dropdown_fund_querytrade_change, names = "value")
dropdown_querytime_querytrade.observe(on_dropdown_querytime_querytrade_change, names = "value")
button_clear_querytrade.on_click(on_button_clear_querytrade)
button_querytrade_summary.on_click(on_button_querytrade_summary)
button_querytrade_summary_all.on_click(on_button_querytrade_summary_all)
# 查询可融资标的券列表
button_501005.on_click(on_button_501005)
dropdown_fund_501005.observe(on_dropdown_fund_501005_change, names = "value")
dropdown_querytime_501005.observe(on_dropdown_querytime_501005, names = "value")
checkbox_501005.observe(on_checkbox_501005, names = "value")
button_clear_501005.on_click(on_button_clear_501005)
# 集中/篮子交易
button_basket.on_click(on_button_basket)
button_basket_filterinstance.on_click(on_button_basket_filterinstance)
button_basket_filterorder.on_click(on_button_basket_filterorder)
button_basket_instance_detail.on_click(on_button_basket_instance_detail)
button_basketpush.on_click(on_button_basketpush)
button_basketpush_all.on_click(on_button_basketpush_all)
dropdown_basket_orderid.observe(on_dropdown_basket_orderid_change, names = "value")
button_clear_basket.on_click(on_button_clear_basket)
button_clear_basket_all.on_click(on_button_clear_basket_all)
# 算法交易
button_algorithm.on_click(on_button_algorithm)
button_algorithm_instance_detail.on_click(on_button_algorithm_instance_detail)
button_algorithm_instance_push.on_click(on_button_algorithm_instance_push)
button_algorithm_order_push.on_click(on_button_algorithm_order_push)
button_clear_algorithm.on_click(on_button_clear_algorithm)
button_clear_algorithm_push.on_click(on_button_clear_algorithm_push)
button_clear_algorithm_all.on_click(on_button_clear_algorithm_all)
button_query_algorithm.on_click(on_button_query_algorithm)
dropdown_querytime_algorithm.observe(on_dropdown_querytime_algorithm_change, names = "value")
# 条件交易
button_condition.on_click(on_button_condition)
button_condition_filterinstance.on_click(on_button_condition_filterinstance)
button_condition_filterorder.on_click(on_button_condition_filterorder)
button_initreqs_condition.on_click(on_button_initreqs_condition)
button_clear_initreqs_condition.on_click(on_button_clear_initreqs_condition)
checkbox_fullinitreqs_condition.observe(on_checkbox_fullinitreqs_condition, names = "value")
button_condition_instance_detail.on_click(on_button_condition_instance_detail)
button_condition_order_detail.on_click(on_button_condition_order_detail)
button_condition_security_order_detail.on_click(on_button_condition_security_order_detail)
button_query_condition.on_click(on_button_query_condition)
button_clear_condition_all.on_click(on_button_clear_condition_all)
dropdown_querytime_condition.observe(on_dropdown_querytime_condition_change, names = "value")

# 设置控件
checkbox_unlimit_table_length.observe(on_checkbox_unlimit_table_length, names = "value")

#******************************************************************************* 

display(Markdown("## <center> 客户端日志分析工具 </center>"))
display(widgets.Output(layout = {"border":"2px solid black", "margin":"0px 0 20px 0"}))
display(widgets.HBox([wind_fileupload, wind_button_analyse, button_init, output_err]))
display(widgets.HBox([output_fundtoken_dict, output_fund_dict]))
display(widgets.HBox([button_summary, output_summary]))
display(widgets.Output(layout = {"border":"2px solid black", "margin":"0px 0 20px 0"}))
display(widgets.HBox([checkbox_unlimit_table_length]))
# 查账户列表区域
display(widgets.HBox([button_account, dropdown_querytime_account, button_clear_account]))
display(widgets.HBox([output_account]))
# 查资金区域
display(widgets.HBox([button_fund, dropdown_fundtoken, button_clear_fund]))
display(widgets.HBox([output_fund]))
# 查持仓区域
display(widgets.HBox([button_position, dropdown_fund_position, dropdown_querytime_position, button_clear_position]))
display(widgets.HBox([output_position]))
# 查询委托
display(widgets.HBox([button_queryorder, dropdown_fund_queryorder, dropdown_querytime_queryorder, button_clear_queryorder]))
display(widgets.HBox([empty_output, button_queryorder_summary, button_queryorder_summary_all]))
display(widgets.HBox([output_queryorder]))
# 查询成交
display(widgets.HBox([button_querytrade, dropdown_fund_querytrade, dropdown_querytime_querytrade, button_clear_querytrade]))
display(widgets.HBox([empty_output, button_querytrade_summary, button_querytrade_summary_all, checkbox_querytrade_stockfilter, input_code_querytrade]))
display(widgets.HBox([output_querytrade]))
# 查询可融资标的券列表 501005
display(widgets.HBox([button_501005, dropdown_fund_501005, dropdown_querytime_501005, button_clear_501005]))
display(widgets.HBox([empty_output, input_code_501005, checkbox_501005]))
display(widgets.HBox([output_501005]))
# 集中/篮子交易区域 basket
display(widgets.HBox([button_basket, input_basket_instanceid, input_code_basket, button_basket_filterinstance, button_basket_filterorder]))
display(widgets.HBox([empty_output, button_basket_instance_detail, button_basketpush_all, input_fund_basket]))
# display(widgets.HBox([empty_output, dropdown_basket_orderid, button_basketpush]))
display(widgets.HBox([empty_output, button_clear_basket, button_clear_basket_all]))
display(widgets.HBox([output_basket]))
display(widgets.HBox([output_basketfilter]))
display(widgets.HBox([output_basketpush]))
# 算法交易区域 algorithm
display(widgets.HBox([button_algorithm, input_algorithm_instanceid, button_query_algorithm, dropdown_querytime_algorithm]))
display(widgets.HBox([empty_output, button_algorithm_instance_detail, button_algorithm_instance_push, button_algorithm_order_push]))
display(widgets.HBox([empty_output, button_clear_algorithm, button_clear_algorithm_push, button_clear_algorithm_all]))
display(widgets.HBox([output_algorithm]))
display(widgets.HBox([output_algorithm_filter]))
display(widgets.HBox([output_algorithm_instance_push]))
display(widgets.HBox([output_algorithm_order_push]))
# 条件交易区域 condition
display(widgets.HBox([button_condition, input_condition_instanceid, button_query_condition, dropdown_querytime_condition]))
display(widgets.HBox([
    empty_output, button_initreqs_condition, checkbox_fullinitreqs_condition, button_clear_initreqs_condition,
    input_condition_fund, input_condition_stockcode],
    layout=widgets.Layout(
        margin='0px',          # 去掉外边距
        padding='0px',         # 去掉内边距
        grid_gap='5px'         # 设置控件之间的间距（如 5px）
    )
))
display(widgets.HBox([
    empty_output, button_condition_instance_detail, button_condition_order_detail, button_condition_security_order_detail,
    button_condition_filterinstance, button_condition_filterorder],
))
display(widgets.HBox([empty_output, button_clear_condition_all]))
display(widgets.HBox([output_condition_initreqs]))
display(widgets.HBox([output_condition]))
display(widgets.HBox([output_condition_instance_detail]))
display(widgets.HBox([output_condition_order_detail]))
display(widgets.HBox([output_condition_order_security_detail]))
# 分割线
display(widgets.Output(layout = {"border":"2px solid black", "margin":"0px 0 20px 0"}))
display(widgets.HBox([wind_input_text, wind_button_raw_request, wind_button_clear]))
display(widgets.HBox([output_request_detail]))

## <center> 客户端日志分析工具 </center>

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…